In [1]:
# CELL 1 – Install & imports

# Optional: install dependencies in a fresh environment
# !pip install neo4j certifi

import os
import xml.etree.ElementTree as ET

from neo4j import GraphDatabase
import certifi

In [7]:
# CELL 2 – Configuration (paths, Neo4j)

# Path to your ArchiMate XML model
# Adjust this to where you store the file in your project
XML_PATH = "./sources/Data/BOM.xml"

# Neo4j connection – fill in or use environment variables
NEO4J_URI      = os.getenv("NEO4J_URI", "neo4j+s://fde218db.databases.neo4j.io")
NEO4J_USER     = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "VgkdUn1MfwDO5ad3TdAh2eFzu9Ry0wNjly1QaFpxJK0")
NEO4J_DB       = os.getenv("NEO4J_DB", "neo4j")

# ArchiMate XML namespace
ARCHIMATE_NS = {"a": "http://www.opengroup.org/xsd/archimate/3.0/"}

In [10]:
# CELL 3 – Optional: proxy cleanup + connectivity test (same style as other notebooks)

URI  = NEO4J_URI
AUTH = (NEO4J_USER, NEO4J_PASSWORD)
DB   = NEO4J_DB

def nuke_proxies():
    """Remove proxy env vars so Aura connectivity is not broken."""
    for k in ("HTTPS_PROXY", "HTTP_PROXY", "https_proxy", "http_proxy", "ALL_PROXY", "all_proxy"):
        os.environ.pop(k, None)
    # Optional: avoid routing Aura traffic through proxies
    os.environ.setdefault("NO_PROXY", "databases.neo4j.io,.neo4j.io")

def try_connect(tag: str, uri: str, driver_kwargs=None) -> bool:
    print(f"Testing {tag} → {uri}")
    try:
        kwargs = dict(auth=AUTH)
        if driver_kwargs:
            kwargs.update(driver_kwargs)
        drv = GraphDatabase.driver(uri, **kwargs)
        drv.verify_connectivity()
        with drv.session(database=DB) as s:
            s.run("RETURN 1").consume()
        print("OK ✅")
        drv.close()
        return True
    except Exception as e:
        print(f"Failed: {e}")
        return False

nuke_proxies()
ok = try_connect("neo4j+s (default trust)", URI)
if not ok:
    print("⚠️ Connectivity test failed. Check URI/credentials/proxy.")
else:
    print("All good. Continuing.")


Testing neo4j+s (default trust) → neo4j+s://fde218db.databases.neo4j.io
OK ✅
All good. Continuing.


In [11]:
# CELL 4 – Parse ArchiMate BOM XML into Python structures

def parse_bom_xml(path: str):
    """Parse ArchiMate 3.0 model and return elements + relationships."""
    tree = ET.parse(path)
    root = tree.getroot()
    ns = ARCHIMATE_NS

    # --- Elements (BusinessObjects, etc.) ---
    elements_node = root.find("a:elements", ns)
    if elements_node is None:
        raise ValueError("No <elements> section found in XML.")

    elements = []
    id_to_name = {}

    for elem in elements_node:
        identifier = elem.attrib.get("identifier")
        # xsi:type is the ArchiMate element type (BusinessObject, etc.)
        archimate_type = None
        for k, v in elem.attrib.items():
            if k.endswith("}type"):
                archimate_type = v
                break

        name_el = elem.find("a:name", ns)
        name = name_el.text.strip() if name_el is not None and name_el.text else None

        if not identifier:
            continue

        record = {
            "identifier": identifier,
            "name": name,
            "archimate_type": archimate_type,  # e.g. "BusinessObject"
        }
        elements.append(record)
        id_to_name[identifier] = name

    # --- Relationships (source/target + label + type) ---
    relationships_node = root.find("a:relationships", ns)
    if relationships_node is None:
        raise ValueError("No <relationships> section found in XML.")

    relationships = []
    for rel in relationships_node:
        identifier = rel.attrib.get("identifier")
        source_id  = rel.attrib.get("source")
        target_id  = rel.attrib.get("target")

        archimate_type = None
        for k, v in rel.attrib.items():
            if k.endswith("}type"):
                archimate_type = v  # e.g. "Association"
                break

        name_el = rel.find("a:name", ns)
        label = name_el.text.strip() if name_el is not None and name_el.text else None

        if not (identifier and source_id and target_id):
            continue

        relationships.append({
            "identifier": identifier,
            "source": source_id,
            "target": target_id,
            "archimate_type": archimate_type,
            "name": label,  # semantic verb, e.g. "contains", "delivers to"
            "source_name": id_to_name.get(source_id),
            "target_name": id_to_name.get(target_id),
        })

    print(f"Parsed {len(elements)} elements and {len(relationships)} relationships from '{path}'.")
    return elements, relationships


elements, relationships = parse_bom_xml(XML_PATH)
elements[:3], relationships[:3]  # quick peek



Parsed 32 elements and 37 relationships from './sources/Data/BOM.xml'.


([{'identifier': 'id-633c0fd5849d41dcb75421563f49c3f5',
   'name': 'Shipment',
   'archimate_type': 'BusinessObject'},
  {'identifier': 'id-629b81aea4fd45e98d40f13a45a476ff',
   'name': 'Driver',
   'archimate_type': 'BusinessObject'},
  {'identifier': 'id-4c8c8ad471704c1198acaf52658eb856',
   'name': 'Route',
   'archimate_type': 'BusinessObject'}],
 [{'identifier': 'id-f44cfda869ec452c96fc0973d40fe29c',
   'source': 'id-9aaacb3448004951b8169828f8c8225c',
   'target': 'id-5cce13aedc0740df89323b83b98f74b4',
   'archimate_type': 'Association',
   'name': 'provides',
   'source_name': 'Ticket',
   'target_name': 'IT System'},
  {'identifier': 'id-329f70da802142659dd180cc32f9d521',
   'source': 'id-5cce13aedc0740df89323b83b98f74b4',
   'target': 'id-3c9186bcb67c4d1ea9f3ceb34c632106',
   'archimate_type': 'Association',
   'name': 'follows',
   'source_name': 'IT System',
   'target_name': 'Policy'},
  {'identifier': 'id-3119e9f07d814532aa5f8db26cfe5295',
   'source': 'id-82ff47cda1bd44088

In [12]:
# CELL 5 – Connect to Neo4j and create basic schema for BOM

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
driver.verify_connectivity()
print("✅ Connected to Neo4j")

with driver.session(database=NEO4J_DB) as session:
    # One unique id per ArchiMate element
    session.run("""
        CREATE CONSTRAINT bomElementId IF NOT EXISTS
        FOR (b:BusinessObject) REQUIRE b.identifier IS UNIQUE;
    """)
    # Optional: also keep name unique for convenience
    session.run("""
        CREATE CONSTRAINT bomElementName IF NOT EXISTS
        FOR (b:BusinessObject) REQUIRE b.name IS UNIQUE;
    """)
    # Optional: relationship identifier uniqueness (if you want)
    session.run("""
        CREATE CONSTRAINT bomRelId IF NOT EXISTS
        FOR ()-[r:ASSOCIATION]-() REQUIRE r.identifier IS UNIQUE;
    """)

print("✅ Constraints created (if not existing).")


✅ Connected to Neo4j
✅ Constraints created (if not existing).


In [13]:
# CELL 6 – Load BusinessObject nodes into Neo4j

def load_business_objects(elements_list):
    with driver.session(database=NEO4J_DB) as session:
        result = session.run(
            """
            UNWIND $rows AS row
            MERGE (b:BusinessObject {identifier: row.identifier})
            SET  b.name           = row.name,
                 b.archimate_type = row.archimate_type
            """,
            rows=elements_list,
        )
    print(f"Upserted {len(elements_list)} BusinessObject nodes.")


load_business_objects(elements)


Upserted 32 BusinessObject nodes.


In [14]:
# CELL 7 – Load relationships into Neo4j

def load_business_relationships(rels_list):
    with driver.session(database=NEO4J_DB) as session:
        session.run(
            """
            UNWIND $rows AS row
            MATCH (src:BusinessObject {identifier: row.source})
            MATCH (tgt:BusinessObject {identifier: row.target})
            MERGE (src)-[r:ASSOCIATION {identifier: row.identifier}]->(tgt)
            SET  r.archimate_type = row.archimate_type,
                 r.name           = row.name
            """,
            rows=rels_list,
        )
    print(f"Upserted {len(rels_list)} relationships.")


load_business_relationships(relationships)


Upserted 37 relationships.


In [15]:
# CELL 8 – Quick sanity checks (optional)

with driver.session(database=NEO4J_DB) as session:
    count_nodes = session.run("MATCH (b:BusinessObject) RETURN count(b) AS c").single()["c"]
    count_rels  = session.run("MATCH ()-[r:ASSOCIATION]->() RETURN count(r) AS c").single()["c"]

print("BusinessObject nodes:", count_nodes)
print("ASSOCIATION relationships:", count_rels)


BusinessObject nodes: 32
ASSOCIATION relationships: 37
